# Chapter 20. 신경망

## 20.0 소개
* 신경망의 핵심은 유닛(노드, 뉴런) 이다.
    * 유닛 : 하나 이상의 입력을 받아 각 입력에 파라미터(가중치)를 곱한다.
    * 가중치가 곱해진 입력에 어떤 bias 값(0)을 더하고 활성화 함수에 이를 전달한다.
    * 출력은 신경망에서 만약 있다면 더 깊은 층에 있는 다른 뉴런을 위해 앞으로 전달한다.
* 피드포워드 신경망 혹은 다층 퍼셉트론 : 다양한 실전 환경에서 사용되는 가장 간단한 인공 신경망이다.
    * 신경망이 일련의 연결된 층으로 표현할 수 있고, 한쪽 끝에는 샘플의 특성값과 다른 한쪽에는 타깃값을 연결한 네트워크 이다.
    * 피드포워드 : 샘플의 특성값이 네트워크 앞쪽으로 주입된다는 사실에서 착안한다.
    * 각 층은 연속적으로 특성값을 변환하여 타깃값과 같은 최종 출력을 내는 게 목적이다.

* 피드 포워드 신경망 : 입력층(input layer), 출력층(output layer), 은닉층(hidden layer)

## 20.1 신경망을 위한 데이터 전처리하기
* StandardScaler로 특성을 표준화한다.

In [6]:
from sklearn import preprocessing
import numpy as np

features = np.array([[-100.1, 3240.1],
                    [-200.2, -234.1],
                    [5000.5, 150.1],
                    [6000.6, -125.1],
                    [9000.9, -673.1]])

scaler = preprocessing.StandardScaler()

# 특성 변환
features_standardized = scaler.fit_transform(features)

# 특성 확인
features_standardized

array([[-1.12541308,  1.96429418],
       [-1.15329466, -0.50068741],
       [ 0.29529406, -0.22809346],
       [ 0.57385917, -0.42335076],
       [ 1.40955451, -0.81216255]])

* 신경망의 모델 파라미터는 작은 난수로 초기화한다.
    * 특성값이 모델 파라미터보다 크면 종종 신경망의 성능이 나빠진다.
    * 샘플의 특성값이 개별 유닛을 통과하면서 합쳐지므로, 모든 특성은 같은 스케일을 가져야 한다.
    * 그래서 각 특성을 모두 평균이 0이고 표준편차 1이되도록 표준화하는 게 가장 좋다.(필수 아니고, 이진 특성인 경우 예외)
    

In [7]:
print("평균:", round(features_standardized[:, 0].mean()))
print("표준편차:", features_standardized[:, 0].std())

평균: 0.0
표준편차: 0.9999999999999999


## 20.2 신경망 구성하기
* 케라스의 Sequential 모델 활용

In [7]:
from keras import models
from keras import layers

# 신경망 모델을 만든다.
network = models.Sequential()

# 렐루 활성화 함수를 사용한 완전 연결층을 추가한다.
network.add(layers.Dense(units=16, activation="relu", input_shape=(10, )))

network.add(layers.Dense(units=16, activation="relu"))

# 시그모이드 활성화 함수를 사용한 완전 연결층 추가
network.add(layers.Dense(units=1, activation="sigmoid"))

# 신경망의 모델 설정 완료
network.compile(loss="binary_crossentropy", # 크로스 엔트로피
optimizer="rmsprop", # 옵티마이저
metrics=["accuracy"]) # 정확도를 성능 지표로 한다.


* 자주 쓰는 조합 확인
    * 이진 분류 : 시그모이드 함수와 하나의 유닛
    * 다중 분류 : 소프트맥스 활성화 함수와 k 개의 유닛(k : 타깃 클래스의 개수)
    * 회귀 : No 활성화 함수
* 손실 함수 정의(예측값이 타깃값과 얼마나 잘 맞는지 측정 함수)
    * 이진 분류 : 이진 크로스 엔트로피
    * 다중 분류 : 범주형 크로스 엔트로피
    * 회귀 : 평균 제곱 오차

* 옵티 마이저 정의 : 가장 작은 손실 함수 오차를 만드는 모델 파라미터 값 찾기
    * 확률적 경사 하강법, 모멘텀(momentum)
    * RMSProp(Root Mean Square Propagation), Adam(Adaptive Moment estimation)
* 하나 이상의 성능 지표(정확도 등)

* 신경망 제작 방법 두 가지 : 층을 쌓는 방식, 함수형 API(functional API)

In [8]:
network.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 16)                176       
_________________________________________________________________
dense_8 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 17        
Total params: 465
Trainable params: 465
Non-trainable params: 0
_________________________________________________________________


* 첫번째 은닉층 : units=16, activation='relu'
    * 렐루 활성화 함수를 가진 16개 유닛 구성
    * 케라스에서 네트워크의 첫 번째 은닉층 : input_shape 파라미터 포함해야 한다.
    * (10,) : 첫 층이 10개의 특성을 가진 샘플을 기대한다.
* 두번째 층 : input_shape 매개변수만 없다. 나머지는 동일하다.
* 출력층 : 이진 분류가 목적이므로 시그모이드 활성화 함수 사용한 유닛 하나만 포함한다.
* 모델 훈련을 위해 케라스에게 네트워크의 훈련 방법 알려준다.
    * compile 메서드 :: 최적화 알고리즘(RMSProp), 손실 함수(binary_crossentropy), 하나 이상의 성능 지표를 지정한다.

* 케라스 summary 메서드 : 추가된 층 수와 가중치 개수를 보여준다.
    * 첫번째 차원은 배치 차원으로, 모델 구성시 None 으로 설정 된다.
    * 160개 가중치 + 16개 절편(bias) = 176개와 일치한다.
    * 첫 입력층은 입력값 자체이며 훈련되는 가중치가 없다. 그래서 은닉층 2개와 출력층 1개만 확인할 수 있다.

In [9]:
# 입력에서 출력까지 3 개의 완전 연결층을 연결한다.
x = layers.Input(shape=(10,))
h1 = layers.Dense(units=16, activation="relu")(x)
h2 = layers.Dense(units=16, activation="relu")(h1)
y = layers.Dense(units=1, activation="sigmoid")(h2)

In [10]:
# 신경망 모델 제작
network = models.Model(x, y)

# 신경망 모델 설정 완료
network.compile(loss="binary_crossentropy",
optimizer="rmsprop",
metrics=["accuracy"])

In [11]:
network.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 16)                176       
_________________________________________________________________
dense_11 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 17        
Total params: 465
Trainable params: 465
Non-trainable params: 0
_________________________________________________________________


* Dense 클래스의 객체 생성 부분 + 입력값 주입 부분 나누어 쓰면 더 쉽게 이해할 수 있다.

In [12]:
dense = layers.Dense(units=16, activation="relu")
h1 = dense(x)

* 파이썬 객체 호출시 특수 메서드 __call__ 실행
    * Dense 층의 정방향 계산(forward propagation) 수행

## 20.3 이진 분류기 훈련하기
* 케라스 사용해서 피드포워드 신경망 만들고 fit 메서드로 훈련

In [19]:
import numpy as np
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers

# 랜덤 시드 설정
np.random.seed(0)

# 필요한 특성 개수를 지정한다.
number_of_features = 1000

# 영화 리뷰 데이터에서 훈련 데이터와 타깃 벡터 로드
(data_train, target_train), (data_test, target_test) = imdb.load_data(
    num_words=number_of_features
)

# 영화 리뷰 데이터를 원-핫 인코딩된 특성 행렬로 변환
tokenizer = Tokenizer(num_words=number_of_features)
features_train = tokenizer.sequences_to_matrix(data_train, mode="binary")
features_test = tokenizer.sequences_to_matrix(data_test, mode="binary")

# 신경망 모델 제작
network = models.Sequential()

# 렐루 활성화 함수를 이용한 완전 연결층 추가
network.add(layers.Dense(units=16, activation="relu", input_shape=(number_of_features,)))

# 렐루 활성화 함수를 이용한 완전 연결층
network.add(layers.Dense(units=16, activation="relu"))

# 시그모이드 활성화 함수를 사용한 완전 연결층 추가
network.add(layers.Dense(units=1, activation="sigmoid"))

# 신경망의 모델 설정 완료
network.compile(loss="binary_crossentropy", # 손실 함수 : 이진 크로스엔트로피
optimizer="rmsprop", # 옵티마이저
metrics=["accuracy"]) # 성능 지표

# 신경망 훈련
history = network.fit(features_train, # 특성
target_train,
epochs=3, # 에폭 횟수
verbose=1, # 에폭 과정을 출력
batch_size = 100, # 배치의 샘플 개수
validation_data=(features_test, target_test) # 테스트 데이터
)



Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 0s 18us/step - loss: 0.4195 - accuracy: 0.8120 - val_loss: 0.3394 - val_accuracy: 0.8546
Epoch 2/3
25000/25000 [==============================] - 0s 14us/step - loss: 0.3225 - accuracy: 0.8641 - val_loss: 0.3284 - val_accuracy: 0.8594
Epoch 3/3
25000/25000 [==============================] - 0s 14us/step - loss: 0.3107 - accuracy: 0.8691 - val_loss: 0.3392 - val_accuracy: 0.8537


In [20]:
# 특성 행렬의 크기 확인

features_train.shape

(25000, 1000)

* epochs : 훈련할 때 사용할 에폭 횟수 정의
* verbose : 훈련 과정 동안 얼마나 많은 정보 출력할지 결정한다.
    * verbose = 0 : 출력 없음
    * verbose=1 : 진행 막대 출력
    * verbose=2 : 에폭당 한 줄씩 로그 출력
* batch_size : 모델 파라미터 업데이트 전에 네트워크 통과시킬 샘플 개수 설정
* 모델 평가 위해 사용할 테스트 세트 보관 : validation_data 변수 활용 혹은 validation_split 파라미터로 평가에 사용할 비율 사전 정의 가능하다.
* 케라스의 fit : 에폭마다 손실값 + 성능 수치 담긴 History 객체 반환(사이킷런은 훈련 모델을 반환)

In [22]:
# 신경망의 모델 설정 완료
network.compile(loss="binary_crossentropy",
optimizer = "rmsprop")

# 신경망 훈련한다.
history = network.fit(features_train,
target_train,
epochs=3,
verbose=1,
batch_size=100,
validation_data = (features_test, target_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 1s 20us/step - loss: 0.3025 - val_loss: 0.3257
Epoch 2/3
25000/25000 [==============================] - 0s 14us/step - loss: 0.2933 - val_loss: 0.3304
Epoch 3/3
25000/25000 [==============================] - 0s 14us/step - loss: 0.2819 - val_loss: 0.3268


In [23]:
network.evaluate(features_test, target_test)

25000/25000 [==============================] - 0s 10us/step


0.3267870804977417

* 케라스 내 IMDB 데이터 : 텍스트를 정수의 리스트로 변환한 것이다.
    * Tokenizer 클래스